In [1]:
!pip install -q --upgrade tensorflow-recommenders tensorflow-datasets

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

user-item matrix 생성

row : user  
column : movie

rate(시청한 것) : c_score

유저가 평점 매긴 것은 본 것으로 간주 -> 잘 안 나옴 / 일정 이상 보지 않은 것은 보지 않은 것으로 간주

In [3]:
ratings = pd.read_csv('c_score_rate.csv')
vod_list = pd.read_csv('vod_list_1205.csv', index_col=0)

In [4]:
ratings[ratings.duplicated()]
vod_list

,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_id,vod_name,running_time
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,0,와이낫크루: 뜻밖의 여행,69분
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,1,그레이트 뷰티,141분
2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,2,해안선,94분
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,3,아는 형님,87분
4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,4,베놈,107분
...,...,...,...,...,...,...,...,...,...,...,...,...
4712,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,13,4712,베놈 2: 렛 데어 비 카니지,97분
4713,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,4713,성월동화 2 - 연전충승,98분
4714,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,4714,다큐인사이트 코드블루,47분
4715,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,4715,캠프 제로베이스원,109분


In [5]:
def create_user_item_matrix(ratings) -> pd.DataFrame:
  mat = ratings.pivot(index="subsr", columns="vod_id", values="ratings")
  # 유저가 평점을 매긴 영화는 '본 영화'로 간주하고 1로 지정합니다
  mat[~mat.isna()] = 1
  # 유저가 평점을 매기지 않은 영화는 NaN인데
  # '보지 않은 영화'로 간주하고 0으로 바꿔줍니다
  mat.fillna(0, inplace=True)
  return mat

user_item_matrix = create_user_item_matrix(ratings)
user_item_matrix

vod_id,0,1,3,5,6,7,8,9,10,11,...,4698,4699,4700,4701,4706,4708,4710,4711,4712,4713
subsr,,,,,,,,,,,,,,,,,,,,,
59879000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59882000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59886000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59890000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59892000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67149000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67154000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67160000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
vod_list

,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_id,vod_name,running_time
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,0,와이낫크루: 뜻밖의 여행,69분
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,1,그레이트 뷰티,141분
2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,2,해안선,94분
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,3,아는 형님,87분
4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,4,베놈,107분
...,...,...,...,...,...,...,...,...,...,...,...,...
4712,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,13,4712,베놈 2: 렛 데어 비 카니지,97분
4713,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,4713,성월동화 2 - 연전충승,98분
4714,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,4714,다큐인사이트 코드블루,47분
4715,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,4715,캠프 제로베이스원,109분


SVD(singular vector decomposition)

In [7]:
def get_svd_prediction(user_item_matrix, k):
    # U, sigma, V 추출
    u, s, vh = scipy.sparse.linalg.svds(user_item_matrix.to_numpy(), k=k)
    # u,s,vh 곱해서 원본 행렬 복원
    preds = np.dot(np.dot(u, np.diag(s)), vh)
    
    # 결과를 DataFrame으로 변환, 정규화
    preds = pd.DataFrame(preds, columns=user_item_matrix.columns, index=user_item_matrix.index)
    preds = (preds - preds.min()) / (preds.max() - preds.min())
    return preds

predictions = get_svd_prediction(user_item_matrix, k=512)
predictions

vod_id,0,1,3,5,6,7,8,9,10,11,...,4698,4699,4700,4701,4706,4708,4710,4711,4712,4713
subsr,,,,,,,,,,,,,,,,,,,,,
59879000,0.004401,0.002164,0.006693,0.018880,0.041713,0.014161,0.026550,1.355324e-15,0.014884,0.008221,...,0.000665,0.003059,0.004705,0.075053,0.001487,0.002977,0.002358,0.001767,0.001487,0.069068
59882000,0.004346,0.002276,0.004716,0.021683,0.025385,0.977609,0.024224,1.041898e-15,0.014192,0.009323,...,0.000494,0.002336,0.004145,0.079141,0.001123,0.004022,0.001263,0.001605,0.001123,0.089020
59886000,0.005074,0.002355,0.008741,0.023626,0.045425,0.021629,0.023616,1.156101e-15,0.016413,0.007550,...,0.000406,0.002212,0.004452,0.072580,0.001693,0.002845,0.002667,0.001458,0.001693,0.061620
59890000,0.002987,0.001409,0.005313,0.011817,0.029503,0.009518,0.031969,1.474305e-15,0.013680,0.009485,...,0.000943,0.003356,0.005774,0.071932,0.001475,0.001880,0.002296,0.001306,0.001475,0.050840
59892000,0.004390,0.002113,0.009864,0.031137,0.066271,0.014715,0.024634,1.077251e-15,0.014482,0.012784,...,0.000790,0.003424,0.005035,0.082399,0.001696,0.002174,0.002222,0.001668,0.001696,0.061066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67149000,0.003547,0.002457,0.006541,0.017521,0.038592,0.012629,0.030475,1.384159e-15,0.012319,0.010980,...,0.000613,0.003101,0.004111,0.085944,0.001661,0.002503,0.002773,0.001738,0.001661,0.073993
67154000,0.005107,0.002311,0.006987,0.025192,0.049170,0.013623,0.028528,1.257567e-15,0.017277,0.007212,...,0.000869,0.003946,0.003225,0.076267,0.001517,0.003392,0.003319,0.001305,0.001517,0.069140
67160000,0.004551,0.003013,0.005071,0.018741,0.034920,0.014267,0.024967,1.322179e-15,0.014576,0.011411,...,0.000495,0.002826,0.006132,0.079049,0.001546,0.003423,0.003325,0.001709,0.001546,0.061052


추천 확인

In [8]:
# 많이 이용한 순으로 유저 목록
ratings.groupby(['subsr']).count().sort_values(by='vod_id', ascending=False).head(20)

,rename,Category,vod_id,Affinity,ratings
subsr,,,,,
60939000,448,448,448,448,448
66705000,232,232,232,232,232
64984000,188,188,188,188,188
64124000,164,164,164,164,164
64659000,153,153,153,153,153
64385000,124,124,124,124,124
60784000,119,119,119,119,119
65696000,96,96,96,96,96
61931000,86,86,86,86,86


In [9]:
# 임의의 유저 시청 목록
subsr = 64659000
subsr_vod_ids = ratings[ratings.subsr == subsr].vod_id  # vod_id 목록
subsr_vod_ids
subsr_vods = vod_list[vod_list.vod_id.isin(subsr_vod_ids)]
print(subsr, '유저가 본 vod 목록')
print(len(subsr_vods), subsr_vods)
print()
# 복원된 행렬에서 유저 row 가져온 후 내림차순으로 정렬
subsr_predictions = predictions.loc[subsr].sort_values(ascending=False)
# 유저가 이미 본 영화는 제외
subsr_predictions = subsr_predictions[~subsr_predictions.index.isin(subsr_vod_ids)]
# 상위 10개 vod 추출
subsr_predictions = subsr_predictions.head(10)
# 10개 vod 정보 가져옴
subsr_recommendations = vod_list[vod_list.vod_id.isin(subsr_predictions.index)]
subsr_recommendations["recommendation_score"] = subsr_predictions.values
print("유저가 보지 않았고 추천할 영화들")
print(subsr_recommendations)
print(subsr_predictions)

64659000 유저가 본 vod 목록
153                     name                                            Summary  \
6             공조2: 인터내셔날  공조 이즈 백! 이번엔 삼각 공조다! 남한으로 숨어든 글로벌 범죄 조직을 잡기 위해...   
14        동상이몽 2 너는 내 운명  다양한 분야의 커플들이 알콩달콩 살아가는 모습을 '남자'와 '여자' 입장에서 바라보...   
38                미운우리새끼  엄마가 화자가 돼 아들의 일상을 관찰하고, 육아일기라는 장치를 통해 순간을 기록하는...   
82          당신이 혹하는 사이 4  5G 광속의 인터넷과 유튜브의 알고리즘을 타고...괴담과도 같은 [미스터리]와 가짜...   
90                 생활의달인  수십년간 한 분야에 종사하며 부단한 열정과 노력으로 달인의 경지에 이르게 된 사람들...   
...                  ...                                                ...   
4450    돌아온 판관 포청천   황금몽  포청천의 절친인 시왕야의 아들, 시옥은 황금을 불법 채굴하여 주나라를 재건하려는 음...   
4594               미인어 2  인어를 연구 중인 해양 연구원 '양범’. 바다에서 난파를 당하지만 정체 모를 여인에...   
4644                오아시스  종두(설경구 분)는 뺑소니 교통 사고로 형을 살다가 교도소에서 막 출소했다. 그 사...   
4706  가족은 괴로워3: 아내여 장미처럼  히라다 가에 도둑이 들어 장남 코노스케의 아내 후미에가 몰래 모아둔 비상금이 사라졌...   
4712    베놈 2: 렛 데어 비 카니지  ‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...   

                         

C:\Users\USER\AppData\Local\Temp\ipykernel_1212\1363786587.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsr_recommendations["recommendation_score"] = subsr_predictions.values


In [10]:
print(subsr, '유저가 본 vod 목록')
print(len(subsr_vods))
subsr_vods

64659000 유저가 본 vod 목록
153


,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_id,vod_name,running_time
6,공조2: 인터내셔날,공조 이즈 백! 이번엔 삼각 공조다! 남한으로 숨어든 글로벌 범죄 조직을 잡기 위해...,"현빈,유해진,임윤아,다니엘 헤니,진선규",이석훈,영화,액션/어드벤쳐,영화,공조2:인터내셔날,30,6,공조2: 인터내셔날,129분
14,동상이몽 2 너는 내 운명,다양한 분야의 커플들이 알콩달콩 살아가는 모습을 '남자'와 '여자' 입장에서 바라보...,"김구라,서장훈,추자현,김수용,이재명","조문주, 강형선, 안병욱, 김미경, 이도희",TV 연예/오락,기타,TV프로그램,동상이몽2너는내운명,115,14,동상이몽 2 - 너는 내 운명,91분
38,미운우리새끼,"엄마가 화자가 돼 아들의 일상을 관찰하고, 육아일기라는 장치를 통해 순간을 기록하는...","신동엽,서장훈","곽승영, 안정현, 정성원 , 김세연, 하나현",TV 연예/오락,기타,TV프로그램,미운우리새끼,275,38,미운 우리 새끼,101분
82,당신이 혹하는 사이 4,5G 광속의 인터넷과 유튜브의 알고리즘을 타고...괴담과도 같은 [미스터리]와 가짜...,"전현무,변영주,봉태규,주우재","박상욱, 김규형, 장경주, 장윤정, 권소현",TV 시사/교양,기타,TV프로그램,당신이혹하는사이4,4,82,당신이 혹하는 사이 4,70분
90,생활의달인,수십년간 한 분야에 종사하며 부단한 열정과 노력으로 달인의 경지에 이르게 된 사람들...,-,"유윤재, 변진선, 이상준, 김동현",TV 연예/오락,기타,TV프로그램,생활의달인,3,90,생활의 달인,67분
...,...,...,...,...,...,...,...,...,...,...,...,...
4450,돌아온 판관 포청천 황금몽,"포청천의 절친인 시왕야의 아들, 시옥은 황금을 불법 채굴하여 주나라를 재건하려는 음...","금초군,하가경,범홍헌",유립립,영화,드라마,영화,돌아온판관포청천황금몽,1,4450,돌아온 판관 포청천 - 황금몽,89분
4594,미인어 2,인어를 연구 중인 해양 연구원 '양범’. 바다에서 난파를 당하지만 정체 모를 여인에...,"임연유,클라라",동위,영화,로맨틱코미디,영화,미인어2,3,4594,미인어2,76분
4644,오아시스,종두(설경구 분)는 뺑소니 교통 사고로 형을 살다가 교도소에서 막 출소했다. 그 사...,"문소리,설경구,안내상,박명신",이창동,영화,멜로,영화,오아시스,1,4644,오아시스,132분
4706,가족은 괴로워3: 아내여 장미처럼,히라다 가에 도둑이 들어 장남 코노스케의 아내 후미에가 몰래 모아둔 비상금이 사라졌...,"하시즈메 이사오,요시유키 카즈코,니시무라 마사히코,나츠카와 유이,나카지마 토모코,히...",야마다 요지,영화,코미디,영화,가족은괴로워3:아내여장미처럼,1,4706,가족은 괴로워3: 아내여 장미처럼,123분


In [11]:
subsr_recommendations

,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_id,vod_name,running_time,recommendation_score
487,미스터트롯2,새로운 전설의 시작 전 국민을 트로트 열풍에 빠지게 만든 레전드의 귀환 대한민국을 ...,"김성주,마스터군단","김상배, 박재용, 전형주, 라수빈, 안영란, 유은혜",TV 연예/오락,기타,TV프로그램,미스터트롯2,42,487,미스터트롯2 - 새로운 전설의 시작,134분,0.129168
549,더시즌즈 악뮤의 오날오밤,"당신의 '모든 고민, 모든 걱정들을 싹 없애줄' 음악과 이야기를 담은 토크쇼",AKMU(악뮤),"조준희, 박석형, 이창수, 서현아, 최지은",TV 연예/오락,기타,TV프로그램,더시즌즈악뮤의오날오밤,4,549,더 시즌즈,79분,0.113117
739,장미꽃 필 무렵,"언제든, 어디든 전국 각지 다양한 팬들을 '장미꽃 필 무렵' 멤버들이 찾아가 노래를...","손태진,신성,에녹,민수현,김중연,박민수,공훈","유웅, 이승훈, 노윤, 박승민, 오성흔, 유지은, 고현정",TV 연예/오락,기타,TV프로그램,장미꽃필무렵,102,739,장미꽃 필 무렵,42분,0.111105
1157,내 딸 금사월,"진실을 숨기려는 자와 복수를 하려는 자, 그들 부부의 25년에 걸친 소리 없는 전쟁...","백진희,윤현민,박세영,도상우,전인화,손창민,박상원,도지원","백호민, 이재진, 김순옥",TV드라마,기타,TV프로그램,내딸금사월,30,1157,"내 딸, 금사월",64분,0.100176
1758,함부로 애틋하게,어린 시절 가슴 아픈 악연으로 헤어졌던 두 남녀가 안하무인 '슈퍼갑 톱스타'와 비굴...,"김우빈,수지,임주환,임주은,진경,유오성,정선경,정동환","박현석, 차영훈, 이경희",TV드라마,기타,TV프로그램,함부로애틋하게,1,1758,함부로 애틋하게,60분,0.099640
2269,어부바,부산 해안가 포구의 멋쟁이 ‘어부바호’ 선장 종범은 철없는 동생의 느닷없는 결혼 선...,"정준호,최대철,이엘빈,박정숙,김수경,현우섭,이이만,권혁진",최종학,영화,드라마,영화,어부바,4,2269,어부바,107분,0.098142
2282,강철비2: 정상회담,"북미 평화협정 체결을 위한 대한민국 대통령(정우성), 북한의 최고지도자인 위원장(유...","정우성,곽도원,유연석,앵거스 맥페이든,신정근,류수영,염정아,김용림,김명곤,장광,이재...",양우석,영화,액션/어드벤쳐,영화,강철비2:정상회담,11,2282,강철비2: 정상회담,142분,0.092280
2307,유혹의 한계,불의의 사고로 하루 아침에 남편을 잃게 된 샤론. 내성적인 고등학생 아들 앤서니가 ...,"크리스티나 크레브,태너 뷰캐넌,세바스찬 카바나스,메건 애슐리 브라운,토미 로즈,제레미 킹",데미언 로메이,영화,공포/스릴러,영화,유혹의한계,1,2307,유혹의 한계,88분,0.091828
2869,미션 임파서블: 데드 레코닝 PART ONE,"가장 위험한 작전, 그의 마지막 선택 모든 인류를 위협할 새로운 무기를 추적하게 된...","톰 크루즈,헤일리 앳웰,빙 레임스,사이먼 페그,레베카 퍼거슨,바네사 커비,에사이 모...",크리스토퍼 맥쿼리,영화,액션/어드벤쳐,영화,미션임파서블:데드레코닝PARTONE,25,2869,미션 임파서블: 데드 레코닝 PART ONE,163분,0.091550
4191,포청천 무덤의 비밀,시신이 되살아나 살인을 저지르는 괴안이 발생한 공안현에 도착한 포청천은 조사에 착수...,"정국림,진몽희,원복복,오명,악동봉",유공지,영화,액션/어드벤쳐,영화,포청천무덤의비밀,19,4191,포청천:무덤의 비밀,88분,0.087847


In [12]:
subsr_predictions

vod_id
2869    0.129168
4191    0.113117
487     0.111105
2269    0.100176
2282    0.099640
549     0.098142
1758    0.092280
739     0.091828
2307    0.091550
1157    0.087847
Name: 64659000, dtype: float64

SVD 클래스화

In [13]:
class SVD:
    def __init__(self, ratings, movies, k): 
        user_item_matrix = create_user_item_matrix(ratings)
        self.preds = get_svd_prediction(user_item_matrix, k)
        self.ratings = ratings
        self.movies = movies

    def get_recommendations(self, user_id, top_k=None):
        user_movie_ids = self.ratings[self.ratings.subsr == user_id].vod_id
        user_movies = self.movies[self.movies.vod_id.isin(user_movie_ids)]

        # 복원된 행렬에서 유저 row만 가져온 뒤 내림차순으로 정렬합니다
        user_predictions = self.preds.loc[user_id].sort_values(ascending=False)
        # 이미 유저가 본 영화는 제외합니다
        user_predictions = user_predictions[~user_predictions.index.isin(user_movie_ids)]
        # 10개 영화의 정보를 가져옵니다
        user_recommendations = self.movies[self.movies.vod_id.isin(user_predictions.index)]
        user_recommendations["recommendation_score"] = user_predictions.values
        
        return user_recommendations if top_k is None else user_recommendations.head(top_k)

In [14]:
svd = SVD(ratings, vod_list, 64)

In [17]:
svd.get_recommendations(64659000, top_k=10)

C:\Users\USER\AppData\Local\Temp\ipykernel_1212\1213603100.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values


,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_id,vod_name,running_time,recommendation_score
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,0,와이낫크루: 뜻밖의 여행,69분,1.000000
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,1,그레이트 뷰티,141분,1.000000
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,3,아는 형님,87분,1.000000
5,극한직업,"불철주야 달리고 구르지만 실적은 바닥, 급기야 해체 위기를 맞는 마약반! 더 이상 ...","류승룡,이하늬,진선규,이동휘,공명,신하균,오정세,김의성",이병헌,영화,코미디,영화,극한직업,13,5,극한직업,111분,0.992683
7,육사오,"우연히 1등 당첨 로또를 주운 말년 병장 ‘천우’. 심장이 터질듯한 설렘도 잠시, ...","고경표,이이경,음문석,박세완,이순원,곽동연,김민호",박규태,영화,코미디,영화,육사오,135,7,육사오(6/45),113분,0.927284
8,꼬리에꼬리를무는그날이야기,세 명의 이야기꾼이 각자의 이야기 친구에게 가장 일상적인 공간에서 과거의 사건을 1...,"장도연,장현성,장성규","김재원, 황성준, 백시원",TV 시사/교양,기타,TV프로그램,꼬리에꼬리를무는그날이야기,518,8,꼬리에 꼬리를 무는 그날 이야기,71분,0.834633
9,나쁜녀석들,강력범죄를 저지른 이들을 모아 더 나쁜 악을 소탕하려 하는 강력계 형사와 나쁜 녀석...,"김상중,마동석,박해진,조동혁,강예원,강신일,민지아,황승언","한동화, 한정훈",TV드라마,외화 시리즈,TV프로그램,나쁜녀석들,5,9,나쁜 녀석들 : 악의 도시,58분,0.827426
10,라바 인 뉴욕,"옐로우와 레드, 두 애벌레의 상상초월 코믹스러운 상황으로 웃음을 자아내는 애니메이션","옐로,레드",-,키즈,오락,키즈,라바인뉴욕,5,10,라바 인 뉴욕,5분,0.810549
11,김과장 특별판,"1, 2회 압축 특별판. 군산 나이트클럽 조폭의 뒤를 봐주며 자금을 삥땅치고 살던 ...","남궁민,남상미,준호,정혜성,김원해,김강현,박영규,이일화",-,TV드라마,기타,TV프로그램,김과장특별판,1,11,김과장 특별판,58분,0.809607
12,신랑수업,대한민국 대표 완벽남들이자 '워너비 신랑감'인 출연진들이 '신랑수업'을 받으며 멋진...,"이승철,김찬우,김준수,영탁,김원희,홍현희","서혜승, 임정규, 이현주, 이혜선",TV 연예/오락,기타,TV프로그램,신랑수업,127,12,요즘 남자 라이프 - 신랑수업,88분,0.800241
